In [1]:
import requests
import pandas as pd
import os

from matplotlib import pyplot as plt
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')

import plotly.express as px
import plotly.graph_objects as go

In [2]:
data = pd.read_csv("../mydata/alldata/cleandata/data.csv")
data.index=data.date
data.drop('date',axis=1,inplace=True)

In [3]:
def toprophet(food):
    fooddicc = {}
    date = []
    price = []
    for i, row in data.iterrows():
        if row['product'] == f'{food}':
            date.append(i)
            price.append(row['price'])

        fooddicc['product'] = f'{food}'
        fooddicc['ds'] = date
        fooddicc['y'] = price
    foodframe = pd.DataFrame(fooddicc)
    foodframe.drop(['product'], axis=1, inplace=True) # Deleting columns product and date to prepare them to plot a graph
    
    groupframe = pd.DataFrame(foodframe.groupby(foodframe.ds)["y"].mean()) # Some dates are repeated, so I group by date
    groupframe.sort_values(by='ds')
    groupframe = groupframe.reset_index()
    groupframe ['ds'] = pd.to_datetime(groupframe.ds)
    return groupframe

In [4]:
ajo = toprophet('ajo')
ajo

,ds,y
0,2004-01-11,108.960000
1,2004-04-10,109.800000
2,2004-06-09,107.595000
3,2004-06-12,114.680000
4,2004-08-11,107.740000
...,...,...
769,2021-08-02,189.460000
770,2021-08-03,194.920000
771,2021-10-05,200.836667
772,2021-11-01,199.116667


In [5]:
for i in set(data['product']):
    prophet = toprophet(f'{i}')
    if not os.path.exists("../mydata/per_item/for_prophet"):
        os.makedirs("../mydata/per_item/for_prophet")
    prophet.to_csv(f"../mydata/per_item/for_prophet/prophet_{i}.csv",index=False)